In [1]:
%load_ext autoreload
%autoreload 2

# # Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / "eye-ai-exec"))

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-06-17 08:59:24,056 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-06-17 08:59:24,057 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-06-17 08:59:27,697 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-06-17 08:59:27,697 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [5]:
datasets = [
    '4-YQVM', # Healthy USC data
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Cropping Image",
    workflow_type="Image Cropping"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

2025-06-17 09:00:40,773 - WARNING - File /home/kb_766/eye-ai-exec/notebooks/Sandbox_KB/HUY_crop_multimodal_images_2-N93J.ipynb has been modified since last commit. Consider commiting before executing


Execution RID: https://www.eye-ai.org/id/eye-ai/4-YTWW@33F-HMX7-D1B6

2025-06-17 09:00:41,306 - INFO - Materialize bag 4-YQVM... 
2025-06-17 09:00:41,426 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-06-17 09:00:41,427 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-06-17 09:00:42,992 - INFO - Using cached bag for  4-YQVM Version:1.0.0
2025-06-17 09:00:42,993 - INFO - Loading /data/4-YQVM_9ac33726c539459d10cd6f6c97a6c9c77b0eefd4545f9eb551fbe334a9b59ad7/Dataset_4-YQVM
2025-06-17 09:00:43,248 - INFO - Creating new database for dataset: 4-YQVM in /data/kb_766/EyeAI_working/4-YQVM@33E-T6MD-NKXR.db
2025-06-17 09:00:43,321 - INFO - Downloading assets ...
2025-06-17 09:00:49,270 - INFO - File [/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5] transfer successful. 76.75 MB transferred at 1

In [6]:
print(execution)

caching_dir: /data
_working_dir: /data/kb_766/EyeAI_working
execution_rid: 4-YTWW
workflow_rid: 4-YTWT
asset_paths: {'Execution_Asset': [AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5')]}
configuration: datasets=[DatasetSpec(rid='4-YQVM', materialize=True, version=DatasetVersion(major=1, minor=0, patch=0))] assets=['2-4JR6'] workflow=Workflow(name='Cropping Image', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/513e000f72eeef4e1f4ce4a85826d6bee253591e--/notebooks/Sandbox_KB/HUY_crop_multimodal_images_2-N93J.ipynb', workflow_type='Image Cropping', version=None, description='', rid=None, checksum='dd9ff8fe57e87667bf4e49ed65cc19966cb31cce', is_notebook=True) parameters={} description='Instance of cropping multimodal images.' argv=['/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/kb_766/.local/share/jupyter/runtime/kernel-403d6c52-c

In [7]:
execution._working_dir

PosixPath('/data/kb_766/EyeAI_working')

In [8]:
ds_bag = execution.datasets[0]
crop_image_model =  execution.asset_paths["Execution_Asset"][0]

In [9]:
df = ds_bag.get_table_as_dataframe("Image")

In [10]:
df.rename(columns={'RID': 'RID_Image'}, inplace=True)

In [11]:
from pathlib import Path
working_dir = Path("~/working_dir")
csv_path = execution._working_dir / "df.csv"
df.to_csv(csv_path, index=False)

In [12]:
table_name = 'Image'
EA.find_features(table_name)

[Feature(target_table=Image, feature_name=Annotation, feature_table=Annotation),
 Feature(target_table=Image, feature_name=Image_Diagnosis, feature_table=Image_Diagnosis),
 Feature(target_table=Image, feature_name=Fundus_Laterality, feature_table=Execution_Image_Fundus_Laterality),
 Feature(target_table=Image, feature_name=Fundus_Angle, feature_table=Execution_Image_Fundus_Angle)]

In [13]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [15]:
output = execution._working_dir / "image_svg_output"
output.mkdir(parents= True, exist_ok=True)

In [17]:

from models.vgg19.vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      ds_bag,
      csv_path,
      execution._working_dir / 'output_train.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )
    

2025-06-17 09:06:49.535857: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-17 09:06:49.535899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-17 09:06:49.536697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 09:06:49.542910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 09:06:50.374437: W tensorflow/compiler/tf2

2025-06-17 09:06:51.338677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-17 09:06:51.365143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-17 09:06:51.366815: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

80134624/80134624 [==============================] - 1s 0us/step


2025-06-17 09:06:52,950 - INFO - Processing image /data/4-YQVM_9ac33726c539459d10cd6f6c97a6c9c77b0eefd4545f9eb551fbe334a9b59ad7/Dataset_4-YQVM/data/asset/Image/Multimodal_1268_Multimodal_2670_Multimodal_11469_Left_Field_2.jpg : , trial 1, color channel grey, resize function imgResize_primary
2025-06-17 09:06:54.068990: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 1s 735ms/step


...

2025-06-17 09:19:25,955 - INFO - SVG for 4-YS5P saved.
2025-06-17 09:19:26,015 - INFO - Processing image /data/4-YQVM_9ac33726c539459d10cd6f6c97a6c9c77b0eefd4545f9eb551fbe334a9b59ad7/Dataset_4-YQVM/data/asset/Image/Multimodal_1324_Multimodal_2768_Multimodal_11567_Right_Field_2.jpg : , trial 1, color channel grey, resize function imgResize_primary


1/1 [==============================] - 0s 20ms/step


2025-06-17 09:19:26,614 - INFO - SVG for 4-YS5R saved.
2025-06-17 09:19:26,674 - INFO - Processing image /data/4-YQVM_9ac33726c539459d10cd6f6c97a6c9c77b0eefd4545f9eb551fbe334a9b59ad7/Dataset_4-YQVM/data/asset/Image/Multimodal_1321_Multimodal_2762_Multimodal_11561_Right_Field_2.jpg : , trial 1, color channel grey, resize function imgResize_primary


1/1 [==============================] - 0s 20ms/step


2025-06-17 09:19:27,542 - INFO - SVG for 4-YS5T saved.
2025-06-17 09:19:27,550 - INFO - Number of images in CSV: 251
2025-06-17 09:19:27,551 - INFO - Number of images in output directory: 251
2025-06-17 09:19:27,551 - INFO - Number of cropped images: 251
2025-06-17 09:19:27,552 - INFO - --- 754.6960508823395 seconds ---
2025-06-17 09:19:27,554 - INFO - Successfully run Ml.
2025-06-17 09:19:27,712 - INFO - Algorithm execution ended.


In [18]:
angle_2_only = EA.filter_angle_2(ds_bag)

In [19]:
csv_output  = Path(execution._working_dir / 'output_train.csv')

In [20]:
df_output = pd.read_csv(csv_output)

In [21]:
filtered_resulted_df = df_output[df_output["Image RID"].isin(angle_2_only["RID"])]

In [22]:
cropping_func_map_output = dict(zip(filtered_resulted_df["Image RID"], filtered_resulted_df["Worked Image Cropping Function"]))

In [23]:
import shutil 

saved_svg_names = filtered_resulted_df["Saved SVG Name"].values

bounding_box_file_paths = []
for file_name in os.listdir(output):
    if file_name in saved_svg_names:
        full_path_name = os.path.join(output, file_name)
        
        bounding_box_file = execution.asset_file_path(
            "Fundus_Bounding_Box", full_path_name, 
        )
        bounding_box_file_paths.append(bounding_box_file)

In [25]:
image_rids = []
angle_2_rids = filtered_resulted_df["Image RID"].values

for file_name in os.listdir(output):
    rid = file_name.split("_")[1].split('.')[0]
    if rid in angle_2_rids:
        image_rids.append(rid)
len(image_rids)

214

In [26]:
image_bounding_box_feature_list = []

for image_rid, asset_file_path in zip(image_rids, bounding_box_file_paths):
    expected_name = f"Cropped_{image_rid}.svg"
    actual_name = os.path.basename(asset_file_path)
    assert expected_name == actual_name, f"Expected {expected_name}, got {actual_name}"

    annotation_func = cropping_func_map_output.get(image_rid)
    if annotation_func:
        image_bounding_box_feature_list.append(
            ImageAnnotationFeature(
                Image=image_rid,
                Execution=execution.execution_rid,
                Fundus_Bounding_Box=asset_file_path,
                Annotation_Function=annotation_func,
                Annotation_Type='Optic Nerve',
            )
        )


In [27]:
with execution.execute() as execution:
    execution.add_features(image_bounding_box_feature_list)

2025-06-17 09:21:26,001 - INFO - Start execution  ...
2025-06-17 09:21:26,087 - INFO - Start execution  ...
2025-06-17 09:21:26,146 - INFO - Successfully run Ml.
2025-06-17 09:21:26,184 - INFO - Algorithm execution ended.


In [28]:
execution.upload_execution_outputs()

2025-06-17 09:23:05,506 - INFO - Uploading execution files...
2025-06-17 09:23:05,601 - INFO - Initializing uploader: GenericUploader v1.7.7 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-06-17 09:23:05,602 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-06-17 09:23:05,603 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-06-17 09:23:05,641 - INFO - Scanning files in directory [/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset]...
2025-06-17 09:23:05,650 - INFO - Including file: [/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/deriva-ml/Execution_Metadata/configuration.json].
2025-06-17 09:23:05,650 - INFO - Including file: [/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/deriva-ml/Execution_Metadata/environment_sn

{'deriva-ml/Execution_Metadata': [AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/deriva-ml/Execution_Metadata/configuration.json'),
  AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/deriva-ml/Execution_Metadata/environment_snapshot_20250617_090049.txt')],
 'eye-ai/Fundus_Bounding_Box': [AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/eye-ai/Fundus_Bounding_Box/Cropped_4-YRP6.svg'),
  AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/eye-ai/Fundus_Bounding_Box/Cropped_4-YRP8.svg'),
  AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/eye-ai/Fundus_Bounding_Box/Cropped_4-YRPA.svg'),
  AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/eye-ai/Fundus_Bounding_Box/Cropped_4-YRPE.svg'),
  AssetFilePath('/data/kb_766/EyeAI_working/deriva-ml/execution/4-YTWW/asset/eye-ai/Fundus_Bounding_Box/Cropped_4-YRPG.svg'),
  AssetFilePath('/data/kb_766